In [1]:
# pip install git+https://github.com/IBM/terratorch.git ##used this code to get latest terratorch download

from Decoder_UNet2D import UNet2D
from Encoder_TerraMind import TerraMindEncoder
from DataLoader import BeforeData
from Losses import DiceLoss
from utils import RandomCrop

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import os




INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
encoder = TerraMindEncoder()
decoder = UNet2D()

composed = transforms.Compose([RandomCrop(224)])
dataset = BeforeData(before_dir = os.getcwd()+ "/Images/Before",
                     after_dir = os.getcwd()+ "/Images/After",
                     label_dir = os.getcwd() + "/Images/Label",
                    #  input_size = 224
                     transform = composed
                     )
dataloader = DataLoader(dataset)#, batch_size=1, shuffle=True)#, num_workers=0)

criterion = DiceLoss()
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-3)
sigmoid = nn.Sigmoid() # probably can and should delete this? Should be in Encoder


device = "cuda" if torch.cuda.is_available() else "cpu"
encoder.to(device)
decoder.to(device)

# ---- Training Loop ----
epochs = 10
for epoch in range(epochs):
    encoder.train()
    decoder.train()
    running_loss = 0.0

    # for sample in dataloader:
        # x_before, x_after, y = sample['x_before'].to(device), sample['x_after'].to(device), sample['y'].to(device)

    for (x_before, x_after), y in dataloader:
        print("Y shape:", y.shape)
        x_before, x_after, y = x_before.to(device), x_after.to(device), y.to(device)

        # Forward pass (explicit encoder + decoder)
        z_before = encoder(x_before)
        z_after = encoder(x_after)
        z_differenced = [after - before for before, after in zip(z_before, z_after)]

        logits = decoder(z_differenced) # reconstruction

        print("logits shape:", logits.shape)
        predictions = sigmoid(logits) # should this be moved to encoder?
        print("predictions:", predictions.shape)
        predictions = torch.argmax(predictions, dim=1)
        print("predictions shape:", predictions.shape, "y shape:", y.shape)

        loss = criterion(predictions, y)

        # Backward pass
        optimizer.zero_grad()
        loss.requires_grad = True
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x_before.size(0)

    epoch_loss = running_loss / len(dataset)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}")

Y shape, dataloader: torch.Size([1, 457, 447])
Y shape: torch.Size([1, 1, 224, 224])
logits shape: torch.Size([1, 3, 224, 224])
predictions: torch.Size([1, 3, 224, 224])
predictions shape: torch.Size([1, 224, 224]) y shape: torch.Size([1, 1, 224, 224])
Epoch 1/10 - Loss: -1.1415
Y shape, dataloader: torch.Size([1, 457, 447])
Y shape: torch.Size([1, 1, 224, 224])
logits shape: torch.Size([1, 3, 224, 224])
predictions: torch.Size([1, 3, 224, 224])
predictions shape: torch.Size([1, 224, 224]) y shape: torch.Size([1, 1, 224, 224])
Epoch 2/10 - Loss: -0.9899
Y shape, dataloader: torch.Size([1, 457, 447])
Y shape: torch.Size([1, 1, 224, 224])
logits shape: torch.Size([1, 3, 224, 224])
predictions: torch.Size([1, 3, 224, 224])
predictions shape: torch.Size([1, 224, 224]) y shape: torch.Size([1, 1, 224, 224])
Epoch 3/10 - Loss: -0.9556
Y shape, dataloader: torch.Size([1, 457, 447])
Y shape: torch.Size([1, 1, 224, 224])
logits shape: torch.Size([1, 3, 224, 224])
predictions: torch.Size([1, 3, 2

In [2]:
composed = transforms.Compose([RandomCrop(224)])
dataset = BeforeData(before_dir = os.getcwd()+ "/Images/Before",
                     after_dir = os.getcwd()+ "/Images/After",
                     label_dir = os.getcwd() + "/Images/Label",
                    #  input_size = 224
                     transform = composed
                     )







In [3]:
DataLoader(dataset, batch_size=1, shuffle=True)#, num_workers=0)
